<a href="https://colab.research.google.com/github/Sommish/Data-Science/blob/main/Flask_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# . Create a Flask App that Displays "Hello, World!" on the Homepage

from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return "Hello, World!"

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
# Build a Flask App with Static HTML Pages and Navigate Between Them

#First, create two HTML files (page1.html and page2.html) in a templates folder.

#page1.html

<!DOCTYPE html>
<html>
<head>
    <title>Page 1</title>
</head>
<body>
    <h1>This is Page 1</h1>
    <a href="/page2">Go to Page 2</a>
</body>
</html>

In [ ]:
page2.html

<!DOCTYPE html>
<html>
<head>
    <title>Page 2</title>
</head>
<body>
    <h1>This is Page 2</h1>
    <a href="/page1">Go to Page 1</a>
</body>
</html>

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/page1')
def page1():
    return render_template('page1.html')

@app.route('/page2')
def page2():
    return render_template('page2.html')

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# 3. Develop a Flask App that Uses URL Parameters to Display Dynamic Content
from flask import Flask

app = Flask(__name__)

@app.route('/greet/<name>')
def greet(name):
    return f"Hello, {name}!"

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
# Create a Flask App with a Form that Accepts User Input and Displays It
form.html

<!DOCTYPE html>
<html>
<head>
    <title>Form</title>
</head>
<body>
    <form method="POST">
        <label for="user_input">Enter something:</label>
        <input type="text" id="user_input" name="user_input">
        <button type="submit">Submit</button>
    </form>
</body>
</html>

In [ ]:
app.py

from flask import Flask, render_template, request

app = Flask(__name__)

@app.route('/form', methods=['GET', 'POST'])
def form():
    if request.method == 'POST':
        user_input = request.form['user_input']
        return f"You entered: {user_input}"
    return render_template('form.html')

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
# Implement User Sessions in a Flask App to Store and Display User-Specific Data
set_session.html

<!DOCTYPE html>
<html>
<head>
    <title>Set Session</title>
</head>
<body>
    <form method="POST">
        <label for="user_data">Enter data:</label>
        <input type="text" id="user_data" name="user_data">
        <button type="submit">Submit</button>
    </form>
</body>
</html>

In [ ]:
app.py

from flask import Flask, render_template, request, session, redirect, url_for

app = Flask(__name__)
app.secret_key = 'supersecretkey'

@app.route('/set_session', methods=['GET', 'POST'])
def set_session():
    if request.method == 'POST':
        session['user_data'] = request.form['user_data']
        return redirect(url_for('get_session'))
    return render_template('set_session.html')

@app.route('/get_session')
def get_session():
    user_data = session.get('user_data', 'No data set')
    return f"User data: {user_data}"

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
# Build a Flask App that Allows Users to Upload Files and Display Them on the Website
upload.html

<!DOCTYPE html>
<html>
<head>
    <title>Upload File</title>
</head>
<body>
    <h1>Upload a File</h1>
    <form method="POST" enctype="multipart/form-data">
        <input type="file" name="file">
        <button type="submit">Upload</button>
    </form>
</body>
</html>
uploaded_file.html

<!DOCTYPE html>
<html>
<head>
    <title>Uploaded File</title>
</head>
<body>
    <h1>File Uploaded Successfully</h1>
    <p>Filename: {{ filename }}</p>
</body>
</html>

In [ ]:
app.py

from flask import Flask, render_template, request, redirect, url_for, flash
from werkzeug.utils import secure_filename
import os

app = Flask(__name__)
app.secret_key = 'supersecretkey'
app.config['UPLOAD_FOLDER'] = 'uploads'

# Ensure the upload folder exists
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

@app.route('/upload', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file:
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            return redirect(url_for('uploaded_file', filename=filename))
    return render_template('upload.html')

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return render_template('uploaded_file.html', filename=filename)

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
# Integrate a SQLite Database with Flask to Perform CRUD Operations on a List of Items

items.html

<!DOCTYPE html>
<html>
<head>
    <title>Items</title>
</head>
<body>
    <h1>Items</h1>
    <form method="POST">
        <input type="text" name="name" placeholder="Item name">
        <button type="submit">Add Item</button>
    </form>
    <ul>
        {% for item in items %}
            <li>{{ item.name }} <a href="/update_item/{{ item.id }}">Edit</a> <a href="/delete_item/{{ item.id }}">Delete</a></li>
        {% endfor %}
    </ul>
</body>
</html>

In [ ]:
update_item.html

<!DOCTYPE html>
<html>
<head>
    <title>Update Item</title>
</head>
<body>
    <h1>Update Item</h1>
    <form method="POST">
        <input type="text" name="name" value="{{ item.name }}">
        <button type="submit">Update</button>
    </form>
</body>
</html>
app.py

from flask import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.secret_key = 'supersecretkey'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///items.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

class Item(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), nullable=False)

@app.route('/items', methods=['GET', 'POST'])
def items():
    if request.method == 'POST':
        item_name = request.form['name']
        new_item = Item(name=item_name)
        db.session.add(new_item)
        db.session.commit()
        return redirect(url_for('items'))
    items = Item.query.all()
    return render_template('items.html', items=items)

@app.route('/delete_item/<int:id>')
def delete_item(id):
    item = Item.query.get_or_404(id)
    db.session.delete(item)
    db.session.commit()
    return redirect(url_for('items'))

@app.route('/update_item/<int:id>', methods=['GET', 'POST'])
def update_item(id):
    item = Item.query.get_or_404(id)
    if request.method == 'POST':
        item.name = request.form['name']
        db.session.commit()
        return redirect(url_for('items'))
    return render_template('update_item.html', item=item)

if __name__ == '__main__':
    db.create_all()
    app.run(debug=True)

In [ ]:
# Implement User Authentication and Registration in a Flask App Using Flask-Login
register.html

<!DOCTYPE html>
<html>
<head>
    <title>Register</title>
</head>
<body>
    <h1>Register</h1>
    <form method="POST">
        <input type="text" name="username" placeholder="Username">
        <input type="password" name="password" placeholder="Password">
        <button type="submit">Register</button>
    </form>
</body>
</html>
login.html

<!DOCTYPE html>
<html>
<head>
    <title>Login</title>
</head>
<body>
    <h1>Login</h1>
    <form method="POST">
        <input type="text" name="username" placeholder="Username">
        <input type="password" name="password" placeholder="Password">
        <button type="submit">Login</button>
    </form>
</body>
</html>

In [ ]:
app.py

from flask import Flask, render_template, request, redirect, url_for, flash, session
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user

app = Flask(__name__)
app.secret_key = 'supersecretkey'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)
login_manager = LoginManager(app)
login_manager.login_view = 'login'

class User(UserMixin, db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(120), nullable=False)

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        new_user = User(username=username, password=password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        user = User.query.filter_by(username=username).first()
        if user and user.password == password:
            login_user(user)
            return redirect(url_for('dashboard'))
        else:
            flash('Invalid credentials')
            return redirect(url_for('login'))
    return render_template('login.html')

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('login'))

@app.route('/dashboard')
@login_required
def dashboard():
    return f"Hello, {current_user.username}! Welcome to your dashboard."

if __name__ == '__main__':
    db.create_all()
    app.run(debug=True)

In [ ]:
# Create a RESTful API Using Flask to Perform CRUD Operations on Resources Like Books or Movies

app.py

from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///books.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

class Book(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(120), nullable=False)
    author = db.Column(db.String(120), nullable=False)

@app.route('/api/books', methods=['GET'])
def get_books():
    books = Book.query.all()
    return jsonify([{'id': book.id, 'title': book.title, 'author': book.author} for book in books])

@app.route('/api/books/<int:id>', methods=['GET'])
def get_book(id):
    book = Book.query.get_or_404(id)
    return jsonify({'id': book.id, 'title': book.title, 'author': book.author})

@app.route('/api/books', methods=['POST'])
def add_book():
    data = request.get_json()
    new_book = Book(title=data['title'], author=data['author'])
    db.session.add(new_book)
    db.session.commit()
    return jsonify({'id': new_book.id}), 201

@app.route('/api/books/<int:id>', methods=['PUT'])
def update_book(id):
    data = request.get_json()

In [ ]:
# Create a Real-Time Chat Application Using Flask-SocketIO

chat.html

<!DOCTYPE html>
<html>
<head>
    <title>Chat</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.0.0/socket.io.js"></script>
    <script type="text/javascript" charset="utf-8">
        document.addEventListener('DOMContentLoaded', function() {
            var socket = io.connect('http://' + document.domain + ':' + location.port);
            socket.on('connect', function() {
                socket.send('User has connected!');
            });
            socket.on('message', function(msg) {
                var li = document.createElement('li');
                li.innerHTML = msg;
                document.getElementById('messages').appendChild(li);
            });
            document.getElementById('sendbutton').onclick = function() {
                var message = document.getElementById('myMessage').value;
                socket.send(message);
                document.getElementById('myMessage').value = '';
            };
        });
    </script>
</head>
<body>
    <h1>Chat Room</h1>
    <ul id="messages"></ul>
    <input type="text" id="myMessage">
    <button id="sendbutton">Send</button>
</body>
</html>

In [ ]:
app.py

from flask import Flask, render_template
from flask_socketio import SocketIO, send

app = Flask(__name__)
app.secret_key = 'supersecretkey'
socketio = SocketIO(app)

@app.route('/chat')
def chat():
    return render_template('chat.html')

@socketio.on('message')
def handle_message(msg):
    print(f"Message: {msg}")
    send(msg, broadcast=True)

if __name__ == '__main__':
    socketio.run(app, debug=True)

In [ ]:
# Build a Flask App that Updates Data in Real-Time Using WebSocket Connections

realtime.html

<!DOCTYPE html>
<html>
<head>
    <title>Real-Time Data</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.0.0/socket.io.js"></script>
    <script type="text/javascript" charset="utf-8">
        document.addEventListener('DOMContentLoaded', function() {
            var socket = io.connect('http://' + document.domain + ':' + location.port);
            socket.on('update_data', function(data) {
                document.getElementById('data').innerHTML = data;
            });
        });
    </script>
</head>
<body>
    <h1>Real-Time Data</h1>
    <div id="data">Waiting for data...</div>
</body>
</html>

In [ ]:
app.py

from flask import Flask, render_template
from flask_socketio import SocketIO, send

app = Flask(__name__)
app.secret_key = 'supersecretkey'
socketio = SocketIO(app)

@app.route('/realtime')
def realtime():
    return render_template('realtime.html')

@socketio.on('update_data')
def handle_update_data(data):
    print(f"Data: {data}")
    send(data, broadcast=True)

if __name__ == '__main__':
    socketio.run(app, debug=True)

In [ ]:
# Implement Notifications in a Flask App Using Websockets to Notify Users of Updates

<!DOCTYPE html>
<html>
<head>
    <title>Notifications</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.0.0/socket.io.js"></script>
    <script type="text/javascript" charset="utf-8">
        document.addEventListener('DOMContentLoaded', function() {
            var socket = io.connect('http://' + document.domain + ':' + location.port);
            socket.on('notify', function(notification) {
                var li = document.createElement('li');
                li.innerHTML = notification;
                document.getElementById('notifications').appendChild(li);
            });
        });
    </script>
</head>
<body>
    <h1>Notifications</h1>
    <ul id="notifications"></ul>
</body>
</html>
app.py

from flask import Flask, render_template
from flask_socketio import SocketIO, send

app = Flask(__name__)
app.secret_key = 'supersecretkey'
socketio = SocketIO(app)

@app.route('/notifications')
def notifications():
    return render_template('notifications.html')

@socketio.on('notify')
def handle_notify(notification):
    print(f"Notification: {notification}")
    send(notification, broadcast=True)

if __name__ == '__main__':
    socketio.run(app, debug=True)